In [1]:
import pandas as pd
from scipy.spatial import cKDTree
import numpy as np
from openai import OpenAI
import os


In [3]:
### CHATPGT ANIMAL family ###

def checkAPIKey():
    # Ensure the API key is set
    api_key = os.getenv("OPENAI_API_KEY")
    if api_key is None:
        raise ValueError("OPENAI_API_KEY environment variable is not set")
    else:
        print("API key is set")


## THESE NAMES ARE STILL DOGSHIT
def getCommonName(family, client):
    print(f"🔍 Asking for a name for {family} ...")

    if family == "" or family is None:
        return None

    prompt = f"Given the fish family '{family}', provide the common English name used for this family. Only return the name and nothing else."

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": f"{prompt}"}],
        temperature=0  # Set temperature to 0 for deterministic responses
    )
    animal_name = completion.choices[0].message.content
                                
    print(f"🐟 Got Answer: {animal_name}")
    return animal_name

In [ ]:
### STEP 1: Add colloquial name for the occurences dataset
gbif_occurences_path = r"data_sources\gbif_occurences.csv"

# Read the csv file
gbif_occurences = pd.read_csv(gbif_occurences_path, delimiter='\t')

# Drop columns where all values are NaN
occurences = gbif_occurences.dropna(axis=1, how='all')

# Drop irrelevant columns
occurrences = gbif_occurences[["family", "decimalLatitude", "decimalLongitude"]]

# Get the unique family names
family_names = occurrences['family'].unique()
checkAPIKey()
client = OpenAI()
animal_names = []
for i in range(len(family_names)):
    print(f"🐟 {i}/{len(family_names)}")
    animal_names.append(getCommonName(family_names[i], client))
    
animal_names = [name.capitalize() for name in animal_names]


API key is set
🐟 0/691
🔍 Asking for a name for Holocentridae ...
🐟 Got Answer: Squirrelfish
🐟 1/691
🔍 Asking for a name for Serranidae ...
🐟 Got Answer: Groupers
🐟 2/691
🔍 Asking for a name for Cheloniidae ...
🐟 Got Answer: Sea turtles
🐟 3/691
🔍 Asking for a name for Aulostomidae ...
🐟 Got Answer: Trumpetfish
🐟 4/691
🔍 Asking for a name for Congridae ...
🐟 Got Answer: Congers
🐟 5/691
🔍 Asking for a name for Sphyraenidae ...
🐟 Got Answer: Barracudas
🐟 6/691
🔍 Asking for a name for Myliobatidae ...
🐟 Got Answer: Eagle rays
🐟 7/691
🔍 Asking for a name for Ginglymostomatidae ...
🐟 Got Answer: Nurse sharks
🐟 8/691
🔍 Asking for a name for Ephippidae ...
🐟 Got Answer: Saddleback fish
🐟 9/691
🔍 Asking for a name for Pomacanthidae ...
🐟 Got Answer: Angelfish
🐟 10/691
🔍 Asking for a name for Scaridae ...
🐟 Got Answer: Parrotfish
🐟 11/691
🔍 Asking for a name for Ostraciidae ...
🐟 Got Answer: Boxfish
🐟 12/691
🔍 Asking for a name for Pomacentridae ...
🐟 Got Answer: Damselfish
🐟 13/691
🔍 Asking for 

C:\Users\pietv\AppData\Local\Temp\ipykernel_28648\2906089363.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occurrences['Animal Name'] = occurrences['family'].map(dict(zip(family_names, animal_names)))


In [ ]:
### STEP 2: Map Family Names (=Scientific Names) to Animal Names (=Common Names) in a new DataFrame

# Create a new DataFrame with an generated Animal_ID and their names
family_names_df = pd.DataFrame(columns=['Family ID','Family Name', 'Animal Name'])

# Add the animal names to the animal_names_df; One animal name per family; possible multiple families per animal name (e.g. "Sea Star can be in multiple families, we dont care which specific family it is)")
family_names_df['Animal Name'] = animal_names
family_names_df['Family Name'] = family_names

# Add the Family ID to the animal_names_df
family_names_df['Family ID'] = np.arange(len(family_names_df))

# save the animal_names_df to a csv file for manual inspection and editing
family_names_df.to_csv(r"cleaned_data\family_names_to_Animal_Names_Wrapper.csv", index=False)

'''IMPORTANT

I manually edited the family_names_df and saved it under final data.
ONLY USE THE EDITED FILE FOR THE NEXT STEPS
'''


In [16]:
'''
IMPORTANT!
WHENEVER WE UPDATE FAMILY_TO_ANIMAL TO A NEW VERSION, WE NEED TO UPDATE THE ANIMAL_NAMES_DF AS WELL 
'''
# TODO: UPDATE THIS VERSION FOR EVERY ITERATION
version = "v1"


## Step 3: With the Animal Names Combine the Common Sightings and the occurences, create animal ID and animal name

# Load the manually edited family_to_animal_v_.csv
''' CHANGE THIS TO THE LATEST VERSION OF THE FAMILY_TO_ANIMAL FILE '''
family_names_df = pd.read_csv(f"final_data/family_to_animal_{version}.csv")

# Get the unique animal names
animal_names = family_names_df['Animal Name']

# Createa a new DataFrame with the animal names and their IDs
animal_names_df = pd.DataFrame(columns=['Animal ID', 'Animal Name'])

# Add the animal names to the animal_names_df
animal_names_df['Animal Name'] = animal_names

# Drop duplicates
animal_names_df = animal_names_df.drop_duplicates(subset='Animal Name')

# Get the sightings from the dive sites excel file
dive_sites_excel_file = pd.ExcelFile(r"cleaned_data\dive_sites_data.xlsx")
sightings = pd.read_excel(dive_sites_excel_file, sheet_name='Common Sightings Connections')

# Get the unique animal names
sightings_animal_names = sightings['Common Sightings'].unique()

# Capitalize the animal names
sightings_animal_names = [name.capitalize() for name in sightings_animal_names]

# Add the animal names to the animal_names_df
for animal_name in sightings_animal_names:
    if animal_name not in animal_names_df['Animal Name'].values:
        animal_names_df = pd.concat([animal_names_df, pd.DataFrame({'Animal Name': [animal_name]})], ignore_index=True)

# Drop duplicates
animal_names_df = animal_names_df.drop_duplicates(subset='Animal Name')

# Add the Animal ID to the animal_names_df
animal_names_df['Animal ID'] = np.arange(len(animal_names_df))

# save the animal_names_df to a csv file for manual inspection and editing
animal_names_df.to_csv(f"final_data/animal_names_{version}.csv", index=False)


In [23]:
### STEP 4: Link the Animals to the Dive Sites !!! REQUIRES STEP 3 TO BE DONE !!!
'''
Dive Site ID | Animal ID | Animal Name | Distance to Divespot in m


We have two sources to do this:
1. The Common Sightings: We already have those in the dive_sites_data.xlsx (Sheet: Common Sightings Connections). We just need to add the Animal ID and a Distance of 0
2. The Occurences: We need to find the occurences that are close to the dive sites and add them to the DataFrame. We utilize KDTree and euclidean distance for this.

'''
# Data was loaded in step 3

## 4.1 Link Common Sightings to Dive Sites

# Add column 'Distance to Divespot in m' to the sightings DataFrame, set to 0
sightings['Distance to Divespot in m'] = 0

# Rename Column 'Common Sightings' to 'Animal Name'
sightings = sightings.rename(columns={'Common Sightings': 'Animal Name'})

# Add the Animal ID to the sightings DataFrame
sightings['Animal ID'] = sightings['Animal Name'].map(dict(zip(animal_names_df['Animal Name'], animal_names_df['Animal ID'])))

## 4.2 Link Occurences to Dive Sites
dive_sites = pd.read_excel(dive_sites_excel_file, sheet_name='Dive Sites')

# Rename columns of the dive site dataset to match the occurences dataset
dive_sites = dive_sites.rename(columns={'Latitude': 'decimalLatitude', 'Longitude': 'decimalLongitude'})

# Convert latitude and longitude columns to numeric types
occurrences['decimalLatitude'] = pd.to_numeric(occurrences['decimalLatitude'], errors='coerce')
occurrences['decimalLongitude'] = pd.to_numeric(occurrences['decimalLongitude'], errors='coerce')
dive_sites['decimalLatitude'] = pd.to_numeric(dive_sites['decimalLatitude'], errors='coerce')
dive_sites['decimalLongitude'] = pd.to_numeric(dive_sites['decimalLongitude'], errors='coerce')

# Create coordinate arrays for both datasets
occurrence_coords = occurrences[['decimalLatitude', 'decimalLongitude']].values
dive_sites_coords = dive_sites[['decimalLatitude', 'decimalLongitude']].values

# Create a KDTree for occurrences
occurrence_tree = cKDTree(occurrence_coords)
threshold = 0.01

print(f" Finding occurences for {len(dive_sites_coords)} dive sites ...")
print(f"🔍 Using a threshold of {threshold} degrees ...")

new_sightings_list = []

print(f"🟢 Start merging dive sites with occurrences ...")
for i, row in dive_sites.iterrows():    # iterate over all dive sites ...

    # ... print out progress
    print(f"  {i}/{len(dive_sites)} dive sites processed ...")

    lat = row['decimalLatitude']
    lon = row['decimalLongitude']
    dive_site_id = row['ID']

    # ... query the KDTree to find all occurrences within the threshold
    indices = occurrence_tree.query_ball_point([lat, lon], r=threshold)
    
    # ... get the common names and euclidean distances for these occurrences
    family_names_for_divespot_i = occurrences.iloc[indices]['family'].tolist()  # get the family names for the occurrences
    family_names_for_divespot_i = list(set(family_names_for_divespot_i)) # remove duplicates
    animal_names_for_divespot_i = family_names_df[family_names_df['Family Name'].isin(family_names_for_divespot_i)]['Animal Name'].tolist() # get the animal names for the family names
    animal_ids_for_divespot_i = animal_names_df[animal_names_df['Animal Name'].isin(animal_names_for_divespot_i)]['Animal ID'].tolist()   # get the animal ids for the animal names
    distances = np.sqrt((occurrences.iloc[indices]['decimalLatitude'] - lat)**2 +
                        (occurrences.iloc[indices]['decimalLongitude'] - lon)**2).tolist()
    
    # calculate the distance in m
    distances = [d * 100000 for d in distances]

    # ... update the sightings with the results
    for j in range(len(animal_ids_for_divespot_i)):
        new_sightings_list.append({'ID': dive_site_id, 'Animal ID': animal_ids_for_divespot_i[j], 'Animal Name': animal_names_for_divespot_i[j], 'Distance to Divespot in m': distances[j]})

# Convert the new sightings list to a DataFrame
new_sightings = pd.DataFrame(new_sightings_list)

# Concatenate the new sightings DataFrame with the existing sightings DataFrame
animals_per_dive_site = pd.concat([sightings, new_sightings], ignore_index=True)

# Sort the sightings by ID
animals_per_dive_site = animals_per_dive_site.sort_values(by='ID')

# Save the updated dive sites DataFrame
output_path = f"final_data/final_{version}.xlsx"

# drop rows in occurences that have a family name that is not in the family_names_df
occurrences = occurrences[occurrences['family'].isin(family_names_df['Family Name'])]

# Use ExcelWriter to add a new sheet
with pd.ExcelWriter(output_path, engine='openpyxl', mode='w') as writer:
    dive_sites.to_excel(writer, sheet_name='Dive Sites', index=False)
    animals_per_dive_site.to_excel(writer, sheet_name='Animals_per_Dive_Site', index=False)
    animal_names_df.to_excel(writer, sheet_name='Animals', index=False)
    family_names_df.to_excel(writer, sheet_name='Family_to_Animal', index=False)
    occurrences.to_excel(writer, sheet_name='Occurences (reduced)', index=False)

# Print out first rows of the updated dive site dataset
print(f"💾 Merged Dataset saved to {output_path}")
print(dive_sites.head())




 Finding occurences for 4395 dive sites ...
🔍 Using a threshold of 0.01 degrees ...
🟢 Start merging dive sites with occurrences ...
  0/4395 dive sites processed ...
  1/4395 dive sites processed ...
  2/4395 dive sites processed ...
  3/4395 dive sites processed ...
  4/4395 dive sites processed ...
  5/4395 dive sites processed ...
  6/4395 dive sites processed ...
  7/4395 dive sites processed ...
  8/4395 dive sites processed ...
  9/4395 dive sites processed ...
  10/4395 dive sites processed ...
  11/4395 dive sites processed ...
  12/4395 dive sites processed ...
  13/4395 dive sites processed ...
  14/4395 dive sites processed ...
  15/4395 dive sites processed ...
  16/4395 dive sites processed ...
  17/4395 dive sites processed ...
  18/4395 dive sites processed ...
  19/4395 dive sites processed ...
  20/4395 dive sites processed ...
  21/4395 dive sites processed ...
  22/4395 dive sites processed ...
  23/4395 dive sites processed ...
  24/4395 dive sites processed ...
  2